# setup snowflake
After we cannot use terraform to setup snowflake, I thought that we could use python to do the rest of the setup. We need to make changes to both aws and snowflake to get the connection running. The goal is to achieve this:
- create an external staging area on aws S3
- load an example data set 

### imports and variables

In [103]:
# imports
import boto3
import os
import snowflake.connector
from dotenv import load_dotenv
import pandas as pd

# load .env
load_dotenv()

# set variables
os.environ["AWS_CONFIG_FILE"] = "~/.aws/config_terraform"
os.environ["AWS_SHARED_CREDENTIALS_FILE"] = "~/.aws/cred_terraform"
SNOWFLAKE_ORGANIZATION_NAME = os.getenv('SNOWFLAKE_ORGANIZATION_NAME')
SNOWFLAKE_ACCOUNT_NAME = os.getenv('SNOWFLAKE_ACCOUNT_NAME')
aws_region = os.getenv('AWS_REGION')
aws_profile = os.getenv('AWS_TERRAFORM_USER')

### create connections to aws and snowflake

In [107]:
# create boto3 session
aws_session = boto3.Session(profile_name=aws_profile)

# create iam client
iam_client = aws_session.client('iam')
print("AWS session established, IAM client created")

AWS session established, IAM client created


In [110]:
# create snowflake session
sf_session = snowflake.connector.connect(
    user= os.getenv('SNOWFLAKE_ADMIN'),              
    password= os.getenv('SNOWSQL_PWD'),           
    account= f"{SNOWFLAKE_ORGANIZATION_NAME}-{SNOWFLAKE_ACCOUNT_NAME}",
    role='ACCOUNTADMIN'                   
)
print("Snowflake session established")

Snowflake session established


In [101]:
# would be nice to get the results of snowflake as a pandas df
def get_table(q): 
    # executes a given query and gives back the result as a DataFrame
    cur = sf_session.cursor()
    cur.execute(q)
    desc = cur.description
    content = cur.fetchall()
    columns = []
    for i in desc:
        columns.append(i[0])
    cur.close()
    return pd.DataFrame(content, columns=columns).set_index(columns[0])

### Now let`s snowflake about the bucket
for that we need to
- get the arn from the snowflake role on aws
- get the uri of the s3 bucket
- give those to snowflake to create a storage integration

In [116]:
snowflake_role = iam_client.get_role(RoleName='Snowflake_Role')
snowflake_role_arn = snowflake_role['Role']['Arn']
s3_uri = f"s3://{os.getenv('PROJECT_NAME')}-i{os.getenv('ITERATION')}-data-lake/staging"

In [112]:
# create s3 storage integration
create_s3_integration = f"""
    CREATE STORAGE INTEGRATION s3_storage_integration
    TYPE = EXTERNAL_STAGE
    STORAGE_PROVIDER = 'S3'
    ENABLED = TRUE
    STORAGE_AWS_ROLE_ARN = '{snowflake_role_arn}'
    STORAGE_ALLOWED_LOCATIONS = ('{s3_uri}');"""
try:
    get_table(create_s3_integration)
    print("S3 Storage Integration Created")
except Exception as e:
    print(e)

002002 (42710): SQL compilation error:
Object 'S3_STORAGE_INTEGRATION' already exists.


### now we can get the properties of the integration from snowflake
... and update the role policy on aws accordingly

In [113]:
q ="DESC INTEGRATION s3_storage_integration;"
s3_integration_properties = get_table(q)
s3_integration_properties

,property_type,property_value,property_default
property,,,
ENABLED,Boolean,true,false
STORAGE_PROVIDER,String,S3,
STORAGE_ALLOWED_LOCATIONS,List,s3://lenico-dwh-i1-data-lake/staging,[]
STORAGE_BLOCKED_LOCATIONS,List,,[]
STORAGE_AWS_IAM_USER_ARN,String,arn:aws:iam::881490105466:user/2avs0000-s,
STORAGE_AWS_ROLE_ARN,String,arn:aws:iam::842676020554:role/snowflake_role,
STORAGE_AWS_EXTERNAL_ID,String,ZT70795_SFCRole=2_eZWIrqwDeHBym60yLJteA2y8fh4=,
COMMENT,String,,


In [115]:
STORAGE_AWS_IAM_USER_ARN = s3_integration_properties.loc['STORAGE_AWS_IAM_USER_ARN','property_value']
STORAGE_AWS_EXTERNAL_ID	= s3_integration_properties.loc['STORAGE_AWS_EXTERNAL_ID','property_value']

### now we have the values we need, let's take a look at the json file to update

In [114]:
pprint(snowflake_role,sort_dicts=False)

{'Role': {'Path': '/',
          'RoleName': 'snowflake_role',
          'RoleId': 'AROA4IM3HUVFBYBO7WDYE',
          'Arn': 'arn:aws:iam::842676020554:role/snowflake_role',
          'CreateDate': datetime.datetime(2024, 12, 4, 16, 47, 29, tzinfo=tzutc()),
          'AssumeRolePolicyDocument': {'Version': '2012-10-17',
                                       'Statement': [{'Effect': 'Allow',
                                                      'Principal': {'Service': 's3.amazonaws.com'},
                                                      'Action': 'sts:AssumeRole'}]},
          'Description': 'Updated description for Snowflake role',
          'MaxSessionDuration': 3600,
          'RoleLastUsed': {}},
 'ResponseMetadata': {'RequestId': '5b12d26b-1118-42bd-9c42-f4c2efaa49cc',
                      'HTTPStatusCode': 200,
                      'HTTPHeaders': {'date': 'Wed, 04 Dec 2024 20:06:19 GMT',
                                      'x-amzn-requestid': '5b12d26b-1118-42bd-9c42-f4

In [ ]:
# change the Trusted entities in the AssumeRolePolity
new_assume_role_policy_document = {
            "Version": "2012-10-17",
            "Statement": [
                {
                "Effect": "Allow",
                "Principal": {"AWS": STORAGE_AWS_IAM_USER_ARN},
                "Action": "sts:AssumeRole",
                "Condition": {"StringEquals": {"sts:ExternalId": STORAGE_AWS_EXTERNAL_ID}}
                }
            ]
        }


In [121]:

# Update the AssumeRolePolicy of the Snowflake_Role
try:
    iam_client.update_assume_role_policy(
        RoleName='Snowflake_Role',
        PolicyDocument=json.dumps(new_assume_role_policy_document)
    )
    print(f"AssumeRolePolicy der Rolle {role_name} erfolgreich aktualisiert.")
except Exception as e:
    print(f"Fehler beim Aktualisieren der Rolle: {e}")


AssumeRolePolicy der Rolle snowflake_role erfolgreich aktualisiert.


## now the statement should be updated - aws and snowflake are now connected

In [99]:
iam_client.get_role(
    RoleName=role_name)['Role']['AssumeRolePolicyDocument']


{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Principal': {'AWS': 'arn:aws:iam::881490105466:user/2avs0000-s'},
   'Action': 'sts:AssumeRole',
   'Condition': {'StringEquals': {'sts:ExternalId': 'ZT70795_SFCRole=2_OPCer6Vn//V57ebu2+NqJCP8JuA='}}}]}